# GERADOR DE POEMAS COM IA E FEEDBACK

## Objetivo do Sistema

Este projeto implementa um **sistema completo de geração de poemas** usando IA generativa via API, com as seguintes funcionalidades:

- **Geração de poemas**: Uso de LLM (Large Language Model) via API
- **Feedback do usuário**: Coleta estruturada de avaliação do poema
- **Novo Chat vs Chat Contínuo**: Diferenciação entre iniciar nova conversa ou continuar com histórico
- **Interface interativa**: Widgets no Jupyter para experiência intuitiva
- **Melhoria automática**: Refinamento de prompts baseado no feedback

## Responsabilidades

- **Grupo 1**: Integração com LLM e chamadas à API
- **Grupo 2**: Interface interativa com widgets
- **Grupo 3**: Sistema de feedback e refinamento de prompts

---

In [ ]:
!pip install gradio==5.40.0
!pip install -U -q "google-genai"

In [ ]:
import gradio as gr
from google import genai

# --- Configuração da API do Gemini ---
try:
    client = genai.Client()
    model_name = "gemini-2.5-pro" 
except Exception as e:
    print(f"Erro ao configurar a API do Gemini: {e}")
    print("Certifique-se de ter configurado sua GEMINI_API_KEY corretamente.")
    client = None
    model_name = None

# --- Funções Auxiliares para o Histórico ---
def formatar_historico_choices(historico):
    """Formata o histórico para ser usado no gr.Radio, mostrando o índice e as primeiras linhas."""
    choices = []
    if historico:
        for i, poema in enumerate(historico):
            primeiras_linhas = " ".join(poema.split('\n')[:2]).strip()
            if len(primeiras_linhas) > 60:
                primeiras_linhas = primeiras_linhas[:57] + "..."
                
            choices.append(f"V{i + 1}: {primeiras_linhas}")
            
    return choices


def selecionar_poema_historico(historico, radio_label_selecionado):
    """Atualiza o poema principal com base na seleção do histórico de rádio."""
    if not radio_label_selecionado or not historico:
        return historico[-1] if historico else "", historico[-1] if historico else ""

    try:
        versao_num = int(radio_label_selecionado.split(':')[0][1:])
        indice = versao_num - 1
        
        if 0 <= indice < len(historico):
            poema_selecionado = historico[indice]
            return poema_selecionado, poema_selecionado 
        else:
            return historico[-1], historico[-1]
    except Exception as e:
        print(f"Erro ao selecionar o poema do histórico: {e}")
        return historico[-1], historico[-1]


# --- Função de Backend para Geração/Refinamento ---
def gerar_poema_com_historico(tema, estilo, tamanho, temperatura, poema_atual, refinamentos_list, chat_atual, historico):
    
    temp_normalizada = temperatura / 100.0
    system_instruction = "Você é um poeta de renome, especialista em vários estilos. Sua única saída deve ser o poema (refinado ou gerado) em português, sem explicações, saudações ou markdown adicional."

    prompt = ""
    criar_novo_chat = False
    
    if refinamentos_list and len(refinamentos_list) > 0:
        prompt = f"Poema atual:\n{poema_atual}\n\nAplique as seguintes modificações:\n"
        for i, ref in enumerate(refinamentos_list, 1):
            prompt += f"{i}. {ref['acao']}: {ref['alvo']}"
            if ref['novo_conteudo']:
                prompt += f" - Novo conteúdo: {ref['novo_conteudo']}"
            prompt += "\n"
        criar_novo_chat = False
    else:
        prompt = f"Crie um poema {estilo} sobre: {tema}."
        if tamanho == "Curto (1-2 estrofes)":
            prompt += " O poema deve ter 1 ou 2 estrofes."
        elif tamanho == "Médio (3-4 estrofes)":
            prompt += " O poema deve ter 3 ou 4 estrofes."
        else:
            prompt += " O poema deve ter 5 ou mais estrofes."
        
        if tema != "":
            historico = []
            chat_atual = None
            criar_novo_chat = True
        else:
            return f"Defina um tema", poema_atual, gr.Tabs(selected=1), chat_atual, historico, gr.update(choices=formatar_historico_choices(historico), value=None)


    if tema == "":
        return f"Defina um tema", poema_atual, gr.Tabs(selected=1), chat_atual, historico, gr.update(choices=formatar_historico_choices(historico), value=None)

    if client and model_name:
        try:
            if criar_novo_chat or chat_atual is None:
                chat = client.chats.create(model=model_name, config={
                    "temperature": temp_normalizada,
                    "system_instruction": system_instruction
                })
            else:
                chat = chat_atual

            response = chat.send_message(prompt)

            if response.text:
                novo_poema = response.text
                
                historico.append(novo_poema)
                
                novas_choices = formatar_historico_choices(historico)
                valor_selecionado = novas_choices[-1] if novas_choices else None

                return novo_poema, novo_poema, gr.Tabs(selected=1), chat, historico, gr.update(choices=novas_choices, value=valor_selecionado)
            else:
                novas_choices = formatar_historico_choices(historico)
                valor_selecionado = novas_choices[-1] if novas_choices else None
                return "Não foi possível gerar/refinar. Tente novamente.", poema_atual, gr.Tabs(selected=1), chat_atual, historico, gr.update(choices=novas_choices, value=valor_selecionado)
        except Exception as e:
            error_message = f"Erro na API do Gemini: {e}"
            print(error_message)
            novas_choices = formatar_historico_choices(historico)
            valor_selecionado = novas_choices[-1] if novas_choices else None
            return f"Ocorreu um erro: {error_message}", poema_atual, gr.Tabs(selected=1), chat_atual, historico, gr.update(choices=novas_choices, value=valor_selecionado)

    # API não configurada
    novas_choices = formatar_historico_choices(historico)
    valor_selecionado = novas_choices[-1] if novas_choices else None
    return "API não configurada.", poema_atual, gr.Tabs(selected=1), None, historico, gr.update(choices=novas_choices, value=valor_selecionado)


# --- Funções de Controle ---
def limpar_campos():
    """Limpa campos e estados, incluindo o histórico."""
    return (
        "", 
        "Livre (sem estrutura específica)", 
        "Médio (3-4 estrofes)", 
        70, 
        "", 
        [], 
        None, 
        [], 
        gr.update(choices=[], value=None)
    )

def toggle_refinamentos():
    return gr.Column(visible=True)

def adicionar_refinamento(refinamentos, acao, alvo, novo_conteudo):
    if not alvo:
        return refinamentos, formatar_refinamentos(refinamentos), "Editar", "", "" 

    refinamentos.append({
        'acao': acao,
        'alvo': alvo,
        'novo_conteudo': novo_conteudo
    })

    return refinamentos, formatar_refinamentos(refinamentos), "Editar", "", ""

def limpar_refinamentos():
    return [], "*Nenhum refinamento adicionado ainda*"

def formatar_refinamentos(refinamentos):
    if not refinamentos:
        return "*Nenhum refinamento adicionado ainda*"

    texto = ""
    for i, ref in enumerate(refinamentos, 1):
        texto += f"**{i}. {ref['acao']}**: {ref['alvo']}"
        if ref['novo_conteudo']:
            texto += f" → *{ref['novo_conteudo']}*"
        texto += "\n\n"
    return texto

def voltar_criar():
    return gr.Tabs(selected=0)

# --- FUNÇÃO PARA CONTROLAR A VISIBILIDADE DO CAMPO NOVO CONTEÚDO ---
def toggle_novo_conteudo(acao):
    """
    Controla se o campo 'Novo Conteúdo' deve ser visível ou editável 
    baseado na ação de refinamento (Adicionar/Editar/Remover).
    """
    if acao == "Adicionar" or acao == "Editar":
        # Necessita de novo conteúdo. Tornar visível e interativo.
        return gr.update(visible=True, interactive=True)
    else: # acao == "Remover"
        # Não precisa de novo conteúdo. Tornar invisível.
        return gr.update(visible=False, interactive=False)


# --- CSS Personalizado (AJUSTADO PARA LARGURA E ALTURA) ---
custom_css = """
.gradio-container {
    max-width: 1200px !important;
    margin: auto !important;
}
h1 {
    text-align: center;
    color: #2c3e50;
    font-size: 2.5em;
    margin-bottom: 10px;
}
.subtitle {
    text-align: center;
    color: #7f8c8d;
    font-size: 1.2em;
    margin-bottom: 30px;
}
.generate-btn {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
    border: none !important;
    color: white !important;
    font-size: 1.1em !important;
    padding: 15px 30px !important;
    border-radius: 10px !important;
    font-weight: bold !important;
    transition: transform 0.2s !important;
}
.generate-btn:hover {
    transform: scale(1.05) !important;
}
.limpar-btn {
    background: #95a5a6 !important;
    color: white !important;
    border-radius: 8px !important;
}
.button-row {
    display: flex !important;
    justify-content: space-between !important;
    gap: 10px !important;
}
.refinamento-box {
    border: 2px solid #667eea;
    border-radius: 10px;
    padding: 15px;
    margin: 10px 0;
    background: #f8f9fa;
    /* Garante que o box de refinamento ocupe a largura completa da coluna */
    width: 100%; 
}
.history-container {
    padding: 10px;
    border: 1px solid #ddd;
    border-radius: 10px;
    background: #f0f4f7;
    height: 100%;
    min-height: 400px;
}
.history-radio > label {
    border: 1px solid #bdc3c7;
    border-radius: 8px;
    margin-bottom: 5px;
    padding: 8px;
    display: block;
    cursor: pointer;
    transition: background-color 0.2s;
    background-color: white;
}
.history-radio > label:hover {
    background-color: #ecf0f1;
}
.history-radio .gr-check-radio-label {
    padding: 0;
    margin: 0;
}
.history-radio input[type="radio"]:checked + .gr-check-radio-label {
    background-color: #e8e6f0;
    border-color: #667eea;
    font-weight: bold;
}
.history-radio input[type="radio"] {
    display: none;
}
/* CLASSE PARA FIXAR ALTURA E EVITAR SALTO VERTICAL */
.poema-refinamento-fixo {
    min-height: 800px; 
}
/* Garante que o layout da coluna do poema/refinamento use 100% da largura alocada */
.column.poema-refinamento-fixo {
    box-sizing: border-box;
}
"""


# --- Interface Gradio ---
with gr.Blocks(theme=gr.themes.Soft(), css=custom_css, title="Gerador de Poemas IA") as demo:
    # --- Estados ---
    poema_state = gr.State(value="")
    refinamentos_state = gr.State(value=[])
    chat_state = gr.State(value=None)
    historico_state = gr.State(value=[])

    gr.Markdown("# Gerador de Poemas com IA")
    gr.Markdown("<p class='subtitle'>✨ Transforme suas ideias em versos únicos e inspiradores</p>")

    with gr.Tabs() as tabs:
        # ABA 1: Definição do Tema
        with gr.Tab("📝 Criar Poema", id=0):
            gr.Markdown("### Defina o tema e estilo do seu poema")

            with gr.Row():
                with gr.Column(scale=2):
                    tema_input = gr.Textbox(
                        label="Tema do Poema",
                        placeholder="Ex: A beleza do pôr do sol, saudade, amor impossível...",
                        lines=3,
                        max_lines=5
                    )

                    estilo_input = gr.Dropdown(
                        label="Estilo do Poema",
                        choices=[
                            "Livre (sem estrutura específica)",
                            "Romântico",
                            "Haicai (estilo japonês)",
                            "Soneto",
                            "Moderno",
                            "Clássico",
                            "Melancólico",
                            "Alegre e descontraído"
                        ],
                        value="Livre (sem estrutura específica)"
                    )

                    tamanho_input = gr.Radio(
                        label="Tamanho do Poema",
                        choices=["Curto (1-2 estrofes)", "Médio (3-4 estrofes)", "Longo (5+ estrofes)"],
                        value="Médio (3-4 estrofes)"
                    )

                with gr.Column(scale=1):
                    gr.Markdown("### ⚙️ Configurações Avançadas")

                    temperatura_input = gr.Slider(
                        minimum=0,
                        maximum=100,
                        value=70,
                        step=1,
                        label="Criatividade",
                        info="Maior = mais criativo e variado"
                    )

                    gr.Markdown("---")
                    gr.Markdown("### 💡 Exemplos de Temas")
                    gr.Markdown("""
                    - O som da chuva à noite
                    - Um gato observando a lua
                    - A solidão na cidade grande
                    - Memórias de infância
                    """)

            with gr.Row(elem_classes=["button-row"]):
                gerar_btn = gr.Button("🚀 Gerar Poema", variant="primary", elem_classes=["generate-btn"])
                limpar_btn = gr.Button("🗑️ Limpar", variant="secondary", elem_classes=["limpar-btn", "generate-btn"])

        # ABA 2: Resultado do Poema
        with gr.Tab("📜 Seu Poema", id=1):
            with gr.Row():
                # COLUNA 1: Poema Atual e Refinamentos
                with gr.Column(scale=4, elem_classes=["poema-refinamento-fixo"]):
                    gr.Markdown("### Poema")
                    
                    poema_output = gr.Textbox(
                        label="",
                        lines=15,
                        max_lines=20,
                        show_label=False,
                        interactive=False,
                        placeholder="Seu poema aparecerá aqui após a geração..."
                    )
                    
                    gr.Markdown("---")

                    # Botão para mostrar/ocultar refinamentos
                    with gr.Row():
                        mostrar_refinamentos_btn = gr.Button("➕ Adicionar Refinamentos", variant="secondary")

                    # Container de refinamentos (inicialmente oculto)
                    refinamentos_container = gr.Column(visible=False)

                    with refinamentos_container:
                        gr.Markdown("### ✂️ Refinamentos")

                        # Container dinâmico para múltiplos blocos
                        refinamentos_accordion = gr.Accordion("Refinamentos Adicionados", open=True)

                        with refinamentos_accordion:
                            refinamentos_display = gr.Markdown("*Nenhum refinamento adicionado ainda*")

                        gr.Markdown("#### Adicionar Novo Refinamento")

                        # GRUPO DE ENTRADA DO REFINAMENTO
                        with gr.Group(elem_classes=["refinamento-box"]):
                            with gr.Column():
                                acao_temp = gr.Radio(
                                    label="Ação",
                                    choices=["Adicionar", "Editar", "Remover"],
                                    value="Editar"
                                )
                                alvo_temp = gr.Textbox(
                                    label="Trecho/Alvo",
                                    placeholder="Ex: 'A terceira estrofe', 'A palavra vento'",
                                    lines=1
                                )
                                # O CAMPO NOVO CONTEÚDO SERÁ CONTROLADO POR JS
                                novo_conteudo_temp = gr.Textbox(
                                    label="Novo Conteúdo",
                                    placeholder="Digite o novo texto (visível para Adicionar/Editar)",
                                    lines=2,
                                    visible=True, # Inicia visível, mas a lógica no .change corrige
                                    interactive=True
                                )

                        with gr.Row():
                            adicionar_refinamento_btn = gr.Button("➕ Adicionar à Lista", variant="secondary")
                            limpar_refinamentos_btn = gr.Button("🗑️ Limpar Todos", variant="secondary")

                        refinar_btn = gr.Button("🔄 Aplicar Refinamentos", variant="primary", elem_classes=["generate-btn"])

                    # LINHA DE BOTÕES FINAL
                    with gr.Row():
                        # O botão Criar Novo Poema ocupa todo o espaço da linha
                        novo_btn = gr.Button("✏️ Criar Novo Poema", variant="primary", size="lg") 

                # COLUNA 2: Histórico
                with gr.Column(scale=1, elem_classes=["history-container"]):
                    gr.Markdown("### 📜 Histórico")
                    gr.Markdown("Selecione a versão para torná-la a atual e aplicar refinamentos.")
                    
                    historico_radio = gr.Radio(
                        label="",
                        choices=[],
                        value=None,
                        show_label=False,
                        elem_classes=["history-radio"]
                    )


    # --- Conectar Eventos ---
    
    # CONEXÃO DA LÓGICA DE VISIBILIDADE DO CAMPO NOVO CONTEÚDO
    acao_temp.change(
        fn=toggle_novo_conteudo,
        inputs=[acao_temp],
        outputs=[novo_conteudo_temp]
    )

    # 1. Geração inicial
    gerar_btn.click(
        fn=lambda t, e, tam, temp, p, r, c, h: gerar_poema_com_historico(t, e, tam, temp, p, [], c, h),
        inputs=[tema_input, estilo_input, tamanho_input, temperatura_input, poema_state, refinamentos_state,
                chat_state, historico_state],
        outputs=[poema_output, poema_state, tabs, chat_state, historico_state, historico_radio]
    )

    # 2. Toggle refinamentos
    mostrar_refinamentos_btn.click(
        fn=toggle_refinamentos,
        outputs=[refinamentos_container]
    )

    # 3. Adicionar refinamento à lista
    adicionar_refinamento_btn.click(
        fn=adicionar_refinamento,
        inputs=[refinamentos_state, acao_temp, alvo_temp, novo_conteudo_temp],
        outputs=[refinamentos_state, refinamentos_display, acao_temp, alvo_temp, novo_conteudo_temp]
    )

    # 4. Limpar refinamentos
    limpar_refinamentos_btn.click(
        fn=limpar_refinamentos,
        outputs=[refinamentos_state, refinamentos_display]
    )

    # 5. Aplicar refinamentos
    refinar_btn.click(
        fn=gerar_poema_com_historico,
        inputs=[tema_input, estilo_input, tamanho_input, temperatura_input, poema_state, refinamentos_state,
                chat_state, historico_state],
        outputs=[poema_output, poema_state, tabs, chat_state, historico_state, historico_radio]
    ).then(
        fn=limpar_refinamentos,
        outputs=[refinamentos_state, refinamentos_display]
    )
    
    # 6. Selecionar poema do histórico (Atualiza o poema principal)
    historico_radio.change(
        fn=selecionar_poema_historico,
        inputs=[historico_state, historico_radio],
        outputs=[poema_output, poema_state]
    )

    # 7. Outros botões
    limpar_btn.click(fn=limpar_campos, outputs=[tema_input, estilo_input, tamanho_input, temperatura_input, poema_state,
                                                refinamentos_state, chat_state, historico_state, historico_radio])
    novo_btn.click(fn=voltar_criar, outputs=tabs)

# --- Lançar Interface ---
demo.launch(inline=True, share=True)